### Edgar Moises Hernandez-Gonzalez
#### 20/04/20-02/07/20
#### Clasificacion de imagenes motoras en EEG con CNN-2D
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Escalogramas CWT (3D)
##### Resize proporcional de 495x500

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import time
import pywt
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score, confusion_matrix
# from keras.utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [2]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/Datos/MI-EEG-A09T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/Datos/MI-EEG-A09E.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/Datos/EtiquetasA09E.csv", header=None)

In [3]:
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)

(288, 22000)
(288, 22000)
(288, 1)


In [4]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y_test))

print("n_classes:", n_classes)

n_classes: 4


In [5]:
#codificar etiquetas con valores de 0 a nClases-1
y_test = LabelEncoder().fit_transform(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
# las etiquetas en categorical para poder entrenar data augmentation
# y_test_categorical = to_categorical(y_test)

In [7]:
#unir 22 canales
def unir_escalogramas_vertical(data):
  dim = (500, 495) #ancho, alto
  fs = 250
  escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,0.5)) / (1/fs)
  datos = np.zeros((data.shape[0],495,500))
  temporal = np.zeros((990,1000))
  for i in range(data.shape[0]): #288 muestras
    for j in range(22): #22 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      #sig = sig[124:624]
      # escalograma CWT
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3', sampling_period = (1 / fs))
      temporal[j*45:(j+1)*45, :] = abs(coef)
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [8]:
#llamar a unir_escalogramas_vertical(data)
inicio = time.time()

x_train = unir_escalogramas_vertical(x_train)
x_test = unir_escalogramas_vertical(x_test)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
0
100
200
Tiempo: 244.90277338027954


In [9]:
print(x_test.shape)

(288, 495, 500)


In [10]:
x = []
x.append(np.ceil(np.max(x_train)))
# esta linea solo se ocupa si se va a predecir un modelo creado con CV
# x.append(np.ceil(np.max(x_test)))

In [11]:
#convertir a float
x_test = x_test.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x_test /= np.ceil(np.max(x))

In [12]:
#convertir de 3D a 4D
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(288, 495, 500, 1)


In [13]:
model = load_model('/content/drive/My Drive/BCI-IV-2a/Modelos/CWT09.hdf5')

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 495, 500, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 247, 250, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 247, 250, 16)      2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 123, 125, 16)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 246000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               31488128  
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)              

In [15]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

In [16]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.8128
accuracy: 0.7882


In [17]:
probabilidades = model.predict(x_test)
y_pred = np.argmax(probabilidades, 1) #1=fila

In [18]:
kappa = cohen_kappa_score(y_test, y_pred)
print("kappa: %.4f " % kappa)

kappa: 0.7176 


In [19]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[68  3  1  0]
 [ 7 44 12  9]
 [ 2  9 53  8]
 [ 5  5  0 62]]
